# 1. Assessor and analyst work

## 1.0. Rating and criteria

Please [open this document](https://static.googleusercontent.com/media/guidelines.raterhub.com/en//searchqualityevaluatorguidelines.pdf)
and study chapters 13.0-13.4. Your task will be to assess the organic answers of search engines given the same query.

## 1.1. Explore the page

For the following search engines:
- https://duckduckgo.com/
- https://www.bing.com/
- https://ya.ru/
- https://www.google.com/

Perform the same query: "**How to get from Kazan to Voronezh**".

Discuss with your TA the following:
1. Which elements you may identify at SERP? Ads, snippets, blends from other sources, ...?
2. Where are organic results? How many of them are there?

## 1.2. Rate the results of the search engine

If there are many of you in the group, assess all search engines, otherwise choose 1 or 2. There should be no less than 5 of your for each search engine. Use the scale from the handbook, use 0..4 numerical equivalents for . 

Compute:
- average relevance and standard deviation.
- [Fleiss kappa score](https://en.wikipedia.org/wiki/Fleiss%27_kappa#Worked_example). Use [this implementation](https://www.statsmodels.org/dev/generated/statsmodels.stats.inter_rater.fleiss_kappa.html).
- [Kendall rank coefficient](https://en.wikipedia.org/wiki/Kendall_rank_correlation_coefficient). Use [this implementation](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.kendalltau.html).

Discuss numerical results. Did you agree on the relevance? Did you agree on the rank? What is the difference?

In [1]:
import numpy as np

ranking_data = np.array([
    [4, 4, 4, 3, 4, 2, 2, 1, 1, 0], 
    [4, 3, 4, 3, 3, 2, 1, 1, 1, 1], 
    [3, 4, 4, 4, 4, 3, 2, 1, 1, 1], 
    [4, 4, 4, 4, 3, 2, 2, 1, 1, 0],
    [4, 4, 4, 4, 3, 2, 2, 1, 1, 3]
])

Averages ang standard deviations per item.

In [2]:
import numpy as np

def compute_mean_and_sigma(ranking_data):
    average_relevance = np.mean(ranking_data, axis=0)
    sigma = np.std(ranking_data, axis=0)
    return average_relevance, sigma

def print_relevance_stats(average_relevance, sigma):
    for i, (mean, std) in enumerate(zip(average_relevance, sigma)):
        print(f"relevance {i}: {mean:.2f} +- {std:.3f}")

average_relevance, sigma = compute_mean_and_sigma(ranking_data)
print_relevance_stats(average_relevance, sigma)

relevance 0: 3.80 +- 0.400
relevance 1: 3.80 +- 0.400
relevance 2: 4.00 +- 0.000
relevance 3: 3.60 +- 0.490
relevance 4: 3.40 +- 0.490
relevance 5: 2.20 +- 0.400
relevance 6: 1.80 +- 0.400
relevance 7: 1.00 +- 0.000
relevance 8: 1.00 +- 0.000
relevance 9: 1.00 +- 1.095


Fleiss kappa score

In [3]:
!pip install statsmodels

In [4]:
from statsmodels.stats.inter_rater import aggregate_raters, fleiss_kappa

aggregate, categories = aggregate_raters(ranking_data.T)
print(f"aggregate: {aggregate}")
print(f"categories: {categories}")
print(f"kappa: {fleiss_kappa(aggregate)}")

aggregate: [[0 0 0 1 4]
 [0 0 0 1 4]
 [0 0 0 0 5]
 [0 0 0 2 3]
 [0 0 0 3 2]
 [0 0 4 1 0]
 [0 1 4 0 0]
 [0 5 0 0 0]
 [0 5 0 0 0]
 [2 2 0 1 0]]
categories: [0 1 2 3 4]
kappa: 0.5156081808396124


In [5]:
ranking_data[1]

array([4, 3, 4, 3, 3, 2, 1, 1, 1, 1])

Kendall tau score is pairwise. Compare one to another.

In [19]:
from scipy.stats import kendalltau

for a in ranking_data:
    for b in ranking_data:
        print(kendalltau(a, b))

SignificanceResult(statistic=1.0, pvalue=0.0003088313222505534)
SignificanceResult(statistic=0.8336550215650926, pvalue=0.0031006074932690315)
SignificanceResult(statistic=0.8058665208462562, pvalue=0.004249822778780865)
SignificanceResult(statistic=0.8648648648648649, pvalue=0.001807038191371059)
SignificanceResult(statistic=0.6301961180038697, pvalue=0.024354510892408494)
SignificanceResult(statistic=0.8336550215650926, pvalue=0.0031006074932690315)
SignificanceResult(statistic=1.0, pvalue=0.0004868116335943063)
SignificanceResult(statistic=0.7142857142857143, pvalue=0.012725353282910863)
SignificanceResult(statistic=0.8336550215650926, pvalue=0.0031006074932690315)
SignificanceResult(statistic=0.7606388292556648, pvalue=0.007541335489120731)
SignificanceResult(statistic=0.8058665208462562, pvalue=0.004249822778780865)
SignificanceResult(statistic=0.7142857142857143, pvalue=0.012725353282910863)
SignificanceResult(statistic=1.0, pvalue=0.0004868116335943063)
SignificanceResult(statis

# 2. Engineer work

You will create a bucket of URLs which are relevant for the query **"free cloud git"**. Then you will automate the search procedure using https://serpapi.com/, or https://developers.google.com/custom-search/v1/overview, or whatever.

Then you will compute MRR@10 and Precision@10.

## 2.1. Build your bucket here

In [7]:
rel_bucket = [
    "github.com",
    "gitpod.io",
    "bitbucket.org",
    "gitlab.com",
    "source.cloud.google.com",
    "sourceforge.net",
    "aws.amazon.com/codecommit/",
    "launchpad.net",
]

query = "free git cloud"

## 2.2. Relevance assessment

Write the code to check that the obtained document is relevant (True) or not (False).

In [8]:
def is_rel(resp_url):
    for u in rel_bucket:
        if u in resp_url:
            return True
    return False

## 2.3. Automation

Get search results from the automation tool you use.

In [9]:
api_key = "5aff1ae53da3a991a97d770bf1991833ba30a97d68925ede4cb0003285c727ba"

In [10]:
import requests 

url = f"https://serpapi.com/search.json?q={query}&hl=en&gl=us&google_domain=google.com&api_key={api_key}"
js = requests.get(url).json()

In [11]:
js['organic_results']

[{'position': 1,
  'title': '6 places to host your git repository',
  'link': 'https://opensource.com/article/18/8/github-alternatives',
  'displayed_link': 'https://opensource.com › article › github-alternatives',
  'date': 'Aug 30, 2018',
  'snippet': '6 places to host your git repository · Option 1: GitHub. Seriously, this is a valid option. · Option 2: GitLab. GitLab is probably the leading ...',
  'snippet_highlighted_words': ['git', 'GitLab', 'GitLab'],
  'about_this_result': {'source': {'description': 'opensource.com was first indexed by Google more than 10 years ago',
    'source_info_link': 'https://opensource.com/article/18/8/github-alternatives',
    'security': 'secure',
    'icon': 'https://serpapi.com/searches/644e9ae73d6919be0d75b56d/images/9342f6b9e9a6299ec0f3e90583a7c2ddb9b0429d2f20ee87da265736fe5f9211a40ebd7fe76747a901370ccf7a78181c.png'}},
  'about_page_link': 'https://www.google.com/search?q=About+https://opensource.com/article/18/8/github-alternatives&tbm=ilp&ilps=

In [12]:
rels = []
for result in js["organic_results"]:
    print(result['position'], result['title'])
    print(result['link'])
    print(is_rel(result['link']))
    rels.append(int(is_rel(result['link'])))
    print()

1 6 places to host your git repository
https://opensource.com/article/18/8/github-alternatives
False

2 Gitpod: Always ready to code.
https://www.gitpod.io/
True

3 14 Git Hosting Services Compared | Tower Blog
https://www.git-tower.com/blog/git-hosting-services-compared/
False

4 Bitbucket | Git solution for teams using Jira
https://bitbucket.org/product
True

5 GitLab: The DevSecOps Platform
https://about.gitlab.com/
True

6 Git
https://git-scm.com/
False

7 Best 13 Free Version Control Hosting Software Picks in 2023
https://www.g2.com/categories/version-control-hosting/free
False

8 Top 10 best Git hosting solutions and services in 2021
https://www.devopsschool.com/blog/top-5-git-hosting-solutions/
False

9 Top GitHub Alternatives to Host Your Open Source Projects
https://itsfoss.com/github-alternatives/
False

10 Best free git hosting? : r/git
https://www.reddit.com/r/git/comments/46t07s/best_free_git_hosting/
False



In [13]:
rels

[0, 1, 0, 1, 1, 0, 0, 0, 0, 0]

## 2.4. MRR

Compute MRR:

In [24]:
def mrr(list_of_lists, k=10):
    r = 0
    for l in list_of_lists:
        r += (1 / (k + 1)) if 1 not in l else 1 / (l.index(1) + 1)
        #print(r)
    return r / len(list_of_lists)

mrr([rels])

0.5

## 2.5. Precision
Compute mean precision:

In [25]:
def mean_precision(list_of_lists, k=10):
    p = 0
    for l in list_of_lists:
        p += sum(l) / k
    return p / len(list_of_lists)

mean_precision([rels])

0.3